<a href="https://colab.research.google.com/github/Eduardoarteagal/PMD-Leccion_1/blob/main/Leccion_PDM_1_2P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=fe82bb3dd348f3f19dee6e1317db000c7e826e65386ef3b04967e047a118bc58
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Leccion").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [21]:
#1.- Crea 3 RDD de pares, uno para cada asignatura, con los alumnos y sus notas
rdd_matematicas = sc.textFile("/content/notas_mate.txt").map(lambda linea: linea.split(",")).map(lambda x: (x[0], float(x[1])))
rdd_ingles = sc.textFile("/content/notas_ingles.txt").map(lambda linea: linea.split(",")).map(lambda x: (x[0], float(x[1])))
rdd_fisica = sc.textFile("/content/notas_fisica.txt").map(lambda linea: linea.split(",")).map(lambda x: (x[0], float(x[1])))

In [22]:
#2.- Crea un solo RDD con todas las notas
rdd_notas_generales = rdd_matematicas.union(rdd_ingles).union(rdd_fisica)
rdd_notas_generales.collect()

[('Angel', 6.0),
 ('Maria', 2.0),
 ('Ramon', 4.5),
 ('Jorge', 10.0),
 ('Susana', 9.0),
 ('Anabel', 8.0),
 ('Pedro', 5.0),
 ('Rocio', 6.0),
 ('Carlos', 4.0),
 ('Triana', 3.0),
 ('Andres', 4.0),
 ('Fernando', 5.0),
 ('Leonardo', 1.0),
 ('Oscar', 7.0),
 ('Isabel', 8.0),
 ('Jose Juan', 5.0),
 ('Nicolas', 2.0),
 ('Alejandro', 5.0),
 ('Rosa', 6.0),
 ('Angel', 4.0),
 ('Maria', 6.0),
 ('Ramon', 8.0),
 ('Jorge', 5.0),
 ('Susana', 2.0),
 ('Anabel', 7.0),
 ('Rocio', 4.0),
 ('Carlos', 8.0),
 ('Triana', 4.0),
 ('Andres', 6.0),
 ('Fernando', 7.0),
 ('Leonardo', 4.0),
 ('Oscar', 3.0),
 ('Isabel', 7.0),
 ('Jose Juan', 3.0),
 ('Nicolas', 5.0),
 ('Alejandro', 7.0),
 ('Rosa', 9.0),
 ('Angel', 9.0),
 ('Maria', 3.0),
 ('Ramon', 7.0),
 ('Jorge', 5.0),
 ('Susana', 9.0),
 ('Anabel', 2.0),
 ('Pedro', 2.0),
 ('Rocio', 5.0),
 ('Carlos', 4.0),
 ('Rocio', 7.0),
 ('Triana', 3.0),
 ('Andres', 4.0),
 ('Fernando', 9.0),
 ('Leonardo', 6.0),
 ('Oscar', 5.0),
 ('Isabel', 8.0),
 ('Jose Juan', 3.0),
 ('Nicolas', 7.0),
 ('A

In [23]:
#3.- ¿Cuál es la nota más baja que ha tenido cada alumno?
notas_mas_baja = rdd_notas_generales.reduceByKey(lambda a, b: min(a, b))
notas_mas_baja.collect()

[('Angel', 4.0),
 ('Carlos', 4.0),
 ('Anabel', 2.0),
 ('Jorge', 5.0),
 ('Susana', 2.0),
 ('Andres', 4.0),
 ('Jose Juan', 3.0),
 ('Rocio', 4.0),
 ('Fernando', 5.0),
 ('Oscar', 3.0),
 ('Isabel', 7.0),
 ('Alejandro', 3.0),
 ('Ramon', 4.5),
 ('Pedro', 2.0),
 ('Leonardo', 1.0),
 ('Nicolas', 2.0),
 ('Maria', 2.0),
 ('Triana', 3.0),
 ('Rosa', 6.0)]

In [24]:
#4.- ¿Cuál es la nota media de cada alumno?
suma_notas = rdd_notas_generales.combineByKey(lambda valor: (valor, 1), lambda acumulador, valor: (acumulador[0] + valor, acumulador[1] + 1), lambda acumulador1, acumulador2: (acumulador1[0] + acumulador2[0], acumulador1[1] + acumulador2[1]))
notas_promedio = suma_notas.mapValues(lambda x: x[0] / x[1])
notas_promedio.collect()

[('Angel', 6.333333333333333),
 ('Carlos', 5.333333333333333),
 ('Anabel', 5.666666666666667),
 ('Jorge', 6.666666666666667),
 ('Susana', 6.666666666666667),
 ('Andres', 4.666666666666667),
 ('Jose Juan', 3.6666666666666665),
 ('Rocio', 5.5),
 ('Fernando', 7.0),
 ('Oscar', 5.0),
 ('Isabel', 7.666666666666667),
 ('Alejandro', 5.0),
 ('Ramon', 6.5),
 ('Pedro', 3.5),
 ('Leonardo', 3.6666666666666665),
 ('Nicolas', 4.666666666666667),
 ('Maria', 3.6666666666666665),
 ('Triana', 3.3333333333333335),
 ('Rosa', 7.666666666666667)]

In [25]:
#5.- ¿Cuantos estudiantes suspende cada asignatura? (Suspende con notas menores a 5)
#[('Mates', 7), ('Física', 8), ('Inglés', 7)]
suspensos_mates = rdd_matematicas.filter(lambda x: x[1] < 5).count()
suspensos_ingles = rdd_ingles.filter(lambda x: x[1] < 5).count()
suspensos_fisica = rdd_fisica.filter(lambda x: x[1] < 5).count()

resultado = [('Mates', suspensos_mates), ('Física', suspensos_fisica), ('Inglés', suspensos_ingles)]
print(resultado)

[('Mates', 7), ('Física', 8), ('Inglés', 7)]
